In [1]:
import pandas as pd
import numpy as np
import time
import scipy.sparse as sp
#import pandas as pd
from implicit.als import AlternatingLeastSquares
import math
from  sklearn.metrics import roc_auc_score 

In [2]:
train = pd.read_csv('../data/ml-1m/train_month.csv')
test = pd.read_csv('../data/ml-1m/test_month.csv')


train_sp_matrix = sp.coo_matrix(([1]*len(train), [train.user.values, \
			train.movie.values])).tolil()
item_list = train.movie.unique()
for u in train.user:
	user_hist = train[train.user==u].movie.values
	neg_i = np.random.choice(item_list)
	while neg_i in user_hist:
		neg_i = np.random.choice(item_list)
	train_sp_matrix[u, neg_i] = 0
train_sp_matrix =train_sp_matrix.tocsr()

In [3]:
print('开始构建模型')
model = AlternatingLeastSquares(factors= 16, iterations = 100)
model.fit(train_sp_matrix.transpose()) 
print('模型构建完毕')

开始构建模型


100%|██████████████████████████████████████████████████████████████████████████████| 100.0/100 [00:10<00:00,  9.10it/s]


模型构建完毕


In [14]:
from sklearn.utils import shuffle
test = shuffle(test)

In [15]:
test.head()

,user,movie,rating,timestamp,month,day
3862,1642,2907,4,1011843582,17,638
4019,1646,2271,3,1045482665,30,1027
3904,1642,2280,4,1011840937,17,638
9105,3413,3684,4,975443359,3,216
10391,3969,2595,4,1019228570,20,723


In [21]:
aucs = []
batch_size = 256
batche_num = len(test)//batch_size
count = 0
u_batches = []
i_batches = []
labels = []
predictions = []
def predict(a,b):
    user_factor = model.user_factors[a].reshape((1,16))
    item_factors = model.item_factors[b]
    prediction = np.dot(user_factor, item_factors.transpose())
    return prediction
for u,i in test[['user', 'movie']].values:
    user_hist = train[train.user==u].movie.values
    predictions.append(predict(u,i))
    labels.append(1)
    neg_i = np.random.choice(item_list)
    while neg_i in user_hist:
        neg_i = np.random.choice(item_list)
    predictions.append(predict(u,neg_i))
    labels.append(0)

for batch_i in range(batche_num):
    auc = roc_auc_score(labels[batch_i*batch_size:(batch_i+1)*batch_size],predictions[batch_i*batch_size:(batch_i+1)*batch_size])
    aucs.append(auc)
auc = np.mean(aucs)
print('aucs: %.4f'%auc)

aucs: 0.7986


In [23]:
list(zip(predictions, labels))[:10]

[(array([0.07767198], dtype=float32), 1),
 (array([0.06633984], dtype=float32), 0),
 (array([0.4540145], dtype=float32), 1),
 (array([0.6101998], dtype=float32), 0),
 (array([0.0105827], dtype=float32), 1),
 (array([0.00209082], dtype=float32), 0),
 (array([0.5494294], dtype=float32), 1),
 (array([0.089223], dtype=float32), 0),
 (array([0.0393864], dtype=float32), 1),
 (array([0.00152138], dtype=float32), 0)]

In [27]:
roc_auc_score(labels, predictions)

0.800895140848746

In [10]:
roc_auc_score(labels, predictions)

0.25

In [24]:
test[:6]

,user,movie,rating,timestamp,month,day
3862,1642,2907,4,1011843582,17,638
4019,1646,2271,3,1045482665,30,1027
3904,1642,2280,4,1011840937,17,638
9105,3413,3684,4,975443359,3,216
10391,3969,2595,4,1019228570,20,723
10223,3884,1313,4,973715729,3,196


In [26]:
2907 in train[train.user==1642].movie.values

False